<a href="https://colab.research.google.com/github/google-research/tapas/blob/master/notebooks/tabfact_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 The Google AI Language Team Authors

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2019 The Google AI Language Team Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Running a Tapas fine-tuned checkpoint
---
This notebook shows how to load and make predictions with TAPAS model, which was introduced in the paper: [TAPAS: Weakly Supervised Table Parsing via Pre-training](https://arxiv.org/abs/2004.02349)

# Clone and install the repository


First, let's install the code.

In [1]:
! pip install tapas-table-parsing

     |████████████████████████████████| 195 kB 10.0 MB/s 
     |████████████████████████████████| 711 kB 13.5 MB/s 
     |████████████████████████████████| 58 kB 5.3 MB/s 
     |████████████████████████████████| 352 kB 38.7 MB/s 
     |████████████████████████████████| 1.5 MB 39.9 MB/s 
     |████████████████████████████████| 3.5 MB 68.6 MB/s 
     |████████████████████████████████| 10.1 MB 56.2 MB/s 
     |████████████████████████████████| 516.4 MB 17 kB/s 
     |████████████████████████████████| 3.5 MB 59.4 MB/s 
     |████████████████████████████████| 56 kB 5.5 MB/s 
     |████████████████████████████████| 77 kB 6.9 MB/s 
     |████████████████████████████████| 1.2 MB 54.3 MB/s 
     |████████████████████████████████| 151 kB 69.0 MB/s 
     |████████████████████████████████| 63.1 MB 11 kB/s 
     |████████████████████████████████| 218 kB 24.4 MB/s 
     |████████████████████████████████| 151 kB 66.5 MB/s 
     |████████████████████████████████| 212 kB 64.8 MB/s 
     |██████████████

# Fetch models fom Google Storage

Next we can get pretrained checkpoint from Google Storage. For the sake of speed, this is a medium sized model trained on [TABFACT](https://tabfact.github.io/). Note that best results in the paper were obtained with with a large model.

In [2]:
! gsutil cp "gs://tapas_models/2020_10_07/tapas_tabfact_inter_masklm_medium_reset.zip" "tapas_model.zip" && unzip tapas_model.zip
! mv tapas_tabfact_inter_masklm_medium_reset tapas_model

Copying gs://tapas_models/2020_10_07/tapas_tabfact_inter_masklm_medium_reset.zip...
- [1 files][407.6 MiB/407.6 MiB]                                                
Operation completed over 1 objects/407.6 MiB.                                    
Archive:  tapas_model.zip
   creating: tapas_tabfact_inter_masklm_medium_reset/
  inflating: tapas_tabfact_inter_masklm_medium_reset/bert_config.json  
  inflating: tapas_tabfact_inter_masklm_medium_reset/README.txt  
  inflating: tapas_tabfact_inter_masklm_medium_reset/model.ckpt.index  
  inflating: tapas_tabfact_inter_masklm_medium_reset/model.ckpt.data-00000-of-00001  
  inflating: tapas_tabfact_inter_masklm_medium_reset/vocab.txt  
  inflating: tapas_tabfact_inter_masklm_medium_reset/model.ckpt.meta  


# Imports

In [3]:
import tensorflow.compat.v1 as tf
import os 
import shutil
import csv
import pandas as pd
import IPython

tf.get_logger().setLevel('ERROR')

In [4]:
from tapas.utils import tf_example_utils
from tapas.protos import interaction_pb2
from tapas.utils import number_annotation_utils
import math


# Load checkpoint for prediction

Here's the prediction code, which will create and `interaction_pb2.Interaction` protobuf object, which is the datastructure we use to store examples, and then call the prediction script.

In [5]:
os.makedirs('results/tabfact/tf_examples', exist_ok=True)
os.makedirs('results/tabfact/model', exist_ok=True)
with open('results/tabfact/model/checkpoint', 'w') as f:
  f.write('model_checkpoint_path: "model.ckpt-0"')
for suffix in ['.data-00000-of-00001', '.index', '.meta']:
  shutil.copyfile(f'tapas_model/model.ckpt{suffix}', f'results/tabfact/model/model.ckpt-0{suffix}')

In [8]:
max_seq_length = 512
vocab_file = "tapas_model/vocab.txt"
config = tf_example_utils.ClassifierConversionConfig(
    vocab_file=vocab_file,
    max_seq_length=max_seq_length,
    max_column_id=max_seq_length,
    max_row_id=max_seq_length,
    strip_column_names=False,
    add_aggregation_candidates=False,
)
converter = tf_example_utils.ToClassifierTensorflowExample(config)

def convert_interactions_to_examples(tables_and_queries):
  """Calls Tapas converter to convert interaction to example."""
  for idx, (table, queries) in enumerate(tables_and_queries):
    interaction = interaction_pb2.Interaction()
    for position, query in enumerate(queries):
      question = interaction.questions.add()
      question.original_text = query
      question.id = f"{idx}-0_{position}"
    for header in table[0]:
      interaction.table.columns.add().text = header
    for line in table[1:]:
      row = interaction.table.rows.add()
      for cell in line:
        row.cells.add().text = cell
    number_annotation_utils.add_numeric_values(interaction)
    for i in range(len(interaction.questions)):
      try:
        yield converter.convert(interaction, i)
      except ValueError as e:
        print(f"Can't convert interaction: {interaction.id} error: {e}")
        
def write_tf_example(filename, examples):
  with tf.io.TFRecordWriter(filename) as writer:
    for example in examples:
      writer.write(example.SerializeToString())

def predict(table_data, queries):
  table = [list(map(lambda s: s.strip(), row.split("|"))) 
           for row in table_data.split("\n") if row.strip()]
  examples = convert_interactions_to_examples([(table, queries)])
  write_tf_example("results/tabfact/tf_examples/test.tfrecord", examples)
  write_tf_example("results/tabfact/tf_examples/dev.tfrecord", [])
  
  ! python -m tapas.run_task_main \
    --task="TABFACT" \
    --output_dir="results" \
    --noloop_predict \
    --test_batch_size={len(queries)} \
    --tapas_verbosity="ERROR" \
    --compression_type= \
    --reset_position_index_per_cell \
    --init_checkpoint="tapas_model/model.ckpt" \
    --bert_config_file="tapas_model/bert_config.json" \
    --mode="predict" 2> error


  results_path = "results/tabfact/model/test.tsv"
  all_results = []
  df = pd.DataFrame(table[1:], columns=table[0])
  display(IPython.display.HTML(df.to_html(index=False)))
  print()
  with open(results_path) as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader:
      supported = int(row["pred_cls"])
      all_results.append(supported)
      score = float(row["logits_cls"])
      position = int(row['position'])
      if supported:
        print("> SUPPORTS:", queries[position])
      else:
        print("> REFUTES:", queries[position])
  return all_results

# Predict

In [12]:
result = predict("""
parameter |	before, ( d = 0 ) |	after, ( d = 1 )
κ p p |	14/day |	2/day
κ p c |	2/day |	2/day
κ c c |	5/day |	3/day
κ c s |	2/day |	1/day
κ p s |	2/day |	2/day
μ (covid-19) |	0.1/day |	0.25/day""",["The before and after value of each parameter is different","The parameter P P is 14/day before quarantine","There are 6 unique parameters in table","C S and P S have same after quarantine values"])

is_built_with_cuda: True
is_gpu_available: True
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Training or predicting ...
Evaluation finished after training step 0.


parameter,"before, ( d = 0 )","after, ( d = 1 )"
κ p p,14/day,2/day
κ p c,2/day,2/day
κ c c,5/day,3/day
κ c s,2/day,1/day
κ p s,2/day,2/day
μ (covid-19),0.1/day,0.25/day



> REFUTES: The before and after value of each parameter is different
> SUPPORTS: The parameter P P is 14/day before quarantine
> SUPPORTS: There are 6 unique parameters in table
> REFUTES: C S and P S have same after quarantine values


#Voila! There you have the results.